In [1]:
# Import Data Science Libraries
# import numpy as np
# import pandas as pd
import tensorflow as tf
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Tensorflow Libraries
from tensorflow import keras
from tensorflow.keras import layers, models
from keras_preprocessing.image import ImageDataGenerator

# System libraries
# from pathlib import Path
# import os.path

from data_generation import prepare_data

BATCH_SIZE = 32
TARGET_SIZE = (150, 150)

In [2]:
train_df, val_df, test_df = prepare_data()

Train Shape:  (67708, 2)
Validation Shape:  (16927, 2)
Test Shape:  (5250, 2)


In [3]:
train_generator = ImageDataGenerator(
                        rescale=1./255,
                        rotation_range=30,
                        brightness_range=[0.7,1.3],
                        shear_range=0.1,
                        zoom_range=0.3,
                        width_shift_range=0.2, 
                        height_shift_range=0.2,
                        horizontal_flip=True)

test_generator = ImageDataGenerator(rescale=1./255)

In [4]:
# Generate Training images
train_images = train_generator.flow_from_dataframe(
                        dataframe=train_df,
                        x_col='Filepath',
                        y_col='Label',
                        target_size=TARGET_SIZE,  # all images will be resized to TARGET_SIZE
                        interpolation='nearest',
                        color_mode='rgb',
                        class_mode='categorical',
                        classes=list(train_df['Label'].unique()),
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        seed=175)

Found 67708 validated image filenames belonging to 525 classes.


In [5]:
# Generate Validation images
val_images = test_generator.flow_from_dataframe(
                    dataframe=val_df,
                    x_col='Filepath',
                    y_col='Label',
                    target_size=TARGET_SIZE,  # all images will be resized to TARGET_SIZE
                    interpolation='nearest',
                    color_mode='rgb',
                    class_mode='categorical',
                    classes=list(train_df['Label'].unique()),
                    batch_size=BATCH_SIZE,
                    shuffle=True,
                    seed=42)


Found 16927 validated image filenames belonging to 525 classes.


In [6]:
# Generate test images
test_images = test_generator.flow_from_dataframe(
                    dataframe=test_df,
                    x_col='Filepath',
                    y_col='Label',
                    target_size=TARGET_SIZE,  # all images will be resized to TARGET_SIZE
                    interpolation='nearest',
                    color_mode='rgb',
                    class_mode='categorical',
                    classes=list(train_df['Label'].unique()),
                    shuffle=False,
                    seed=42)

Found 5245 validated image filenames belonging to 525 classes.


In [8]:
model = models.Sequential()
model.add(layers.Conv2D(16, (3, 3), strides=(1, 1), padding='same', activation='relu', input_shape=(TARGET_SIZE[0], TARGET_SIZE[1], 3)))
model.add(layers.MaxPooling2D(pool_size=3, padding="same"))

model.add(layers.Conv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=3, padding="same"))

model.add(layers.Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=3, padding="same"))

model.add(layers.Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=3, padding="same"))

model.add(layers.Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=3, padding="same"))

model.add(layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=3, padding="same"))

# model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(525, activation='softmax'))

opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt,
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

model.summary()

/Users/akhil/Documents/NEU DAE/Spring24/NeuralNets/Project/bird_image_classifier/nndl_env/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 150, 150, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 50, 50, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 50, 50, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 17, 17, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 17, 17, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 6, 6, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 2, 2, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 1, 1, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │        25,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 525)            │        53,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,061,413 (4.05 MB)

 Trainable params: 1,061,413 (4.05 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="./model/cnn_512.keras",
        save_best_only=True,
        monitor="val_loss"), 
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                      factor=0.2, 
                                      patience=3, 
                                      min_lr=1e-6),
    keras.callbacks.EarlyStopping(monitor='val_loss', 
                                  verbose=1,
                                  min_delta=0.01,
                                  patience=5)
                                      ]

history = model.fit(train_images, epochs=100, 
                    validation_data=val_images, callbacks=callbacks)
model.save_weights('./model/cnn_512.weights.h5') 

Epoch 1/100


2024-04-12 12:42:46.025387: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/Users/akhil/Documents/NEU DAE/Spring24/NeuralNets/Project/bird_image_classifier/nndl_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2116/2116 ━━━━━━━━━━━━━━━━━━━━ 214s 100ms/step - accuracy: 0.0038 - loss: 6.1222 - val_accuracy: 0.0145 - val_loss: 5.4941 - learning_rate: 5.0000e-04
Epoch 2/100
2116/2116 ━━━━━━━━━━━━━━━━━━━━ 211s 100ms/step - accuracy: 0.0196 - loss: 5.4352 - val_accuracy: 0.0402 - val_loss: 4.9868 - learning_rate: 5.0000e-04
Epoch 3/100
2116/2116 ━━━━━━━━━━━━━━━━━━━━ 212s 100ms/step - accuracy: 0.0402 - loss: 5.0260 - val_accuracy: 0.0683 - val_loss: 4.6677 - learning_rate: 5.0000e-04
Epoch 4/100
2116/2116 ━━━━━━━━━━━━━━━━━━━━ 214s 101ms/step - accuracy: 0.0772 - loss: 4.5963 - val_accuracy: 0.1310 - val_loss: 4.1491 - learning_rate: 5.0000e-04
Epoch 5/100
2116/2116 ━━━━━━━━━━━━━━━━━━━━ 215s 101ms/step - accuracy: 0.1148 - loss: 4.2746 - val_accuracy: 0.1828 - val_loss: 3.7827 - learning_rate: 5.0000e-04
Epoch 6/100
2116/2116 ━━━━━━━━━━━━━━━━━━━━ 216s 102ms/step - accuracy: 0.1625 - loss: 3.9562 - val_accuracy: 0.2194 - val_loss: 3.6113 - learning_rate: 5.0000e-04
Epoch 7/100
2116/2116 ━━━━━━━━━━━━